In [2]:
import tensorflow as tf

C:\Users\wjssm\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### 상수

In [4]:
a = tf.constant(120, name = 'a')
b = tf.constant(130, name = 'b')
c = tf.constant(140, name = 'c')

### 변수

In [5]:
v = tf.Variable(0, name = 'v')

## assign

In [6]:
calc_op = a+b+c
assign_op = tf.assign(v, calc_op)

### 실행

In [7]:
sess = tf.Session()
sess.run(assign_op)

390

In [8]:
print(sess.run(v))

390


### placeholder

In [9]:
a = tf.placeholder(tf.int32, [3])

In [10]:
b = tf.constant(2)
x2_op = a*b

In [13]:
sess = tf.Session()

In [14]:
r1 = sess.run(x2_op, feed_dict = {a:[1,2,3]}) #placeholder 때문
r1

array([2, 4, 6])

In [15]:
r2 = sess.run(x2_op, feed_dict={a:[10,20,10]})
r2

array([20, 40, 20])

##### - 원하는 크기의 배열을 사용하고 싶을 떄

In [16]:
a = tf.placeholder(tf.int32, [None])

In [17]:
b = tf.constant(10)
x10_op = a*b

In [18]:
r1 = sess.run(x10_op, feed_dict = {a:[1,2,3,4,5]})
print(r1)

[10 20 30 40 50]


# bmi.csv

In [19]:
import numpy as np
import pandas as pd

In [20]:
csv = pd.read_csv('./ex_code/ch5/bmi.csv')

In [22]:
csv.head()

,height,weight,label
0,142,62,fat
1,142,73,fat
2,177,61,normal
3,187,48,thin
4,153,60,fat


###### 데이터 정규화

In [23]:
csv['height'] = csv['height']/200
csv['weight'] = csv['weight']/100

##### label -> 배열
- thin (1,0,0), normal(0,1,0), fat(0,0,1)

In [24]:
bclass = {'thin' : (1,0,0), 'normal' : (0,1,0), 'fat' : (0,0,1)}

In [29]:
bclass['normal']

(0, 1, 0)

In [26]:
csv['label_pat'] = csv['label'].apply(lambda x : np.array(bclass[x]))

In [30]:
csv.head()

,height,weight,label,label_pat
0,0.710,0.62,fat,"[0, 0, 1]"
1,0.710,0.73,fat,"[0, 0, 1]"
2,0.885,0.61,normal,"[0, 1, 0]"
3,0.935,0.48,thin,"[1, 0, 0]"
4,0.765,0.60,fat,"[0, 0, 1]"


In [33]:
from sklearn.model_selection import train_test_split 

In [40]:
X_train, X_test, Y_train, Y_test = train_test_split(csv[['height','weight']],csv['label_pat'],test_size = 0.25)

In [59]:
Y_test = list(Y_test)

### placeholder

In [41]:
x = tf.placeholder(tf.float32, [None,2])
y_ = tf.placeholder(tf.float32, [None,3])

### variable

In [43]:
W = tf.Variable(tf.zeros([2,3]))
b = tf.Variable(tf.zeros([3]))
y = tf.nn.softmax(tf.matmul(x,W) + b)

### model training

In [44]:
cross_entropy = -tf.reduce_sum(y_*tf.log(y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train = optimizer.minimize(cross_entropy)

### 정답률

In [48]:
predict = tf.equal(tf.argmax(y,1) , tf.argmax(y_,1)) #y: predict value, y_ : true value
accuracy = tf.reduce_mean(tf.cast(predict, tf.float32))

### 세션 시작하기

In [49]:
sess.run(tf.global_variables_initializer())

### 학습시키기

In [63]:
for step in range(3500) :
    i = (step * 100) % 14000
    x_pat = X_train[1+i : 1+i+100]
    y_ans = list(Y_train[1+i : 1+i+100])
    
    fd = {x:x_pat, y_:y_ans}
    sess.run(train, feed_dict = fd)
    if step%500 ==0 :
        cre = sess.run(cross_entropy, feed_dict = fd)
        acc = sess.run(accuracy, feed_dict = {x:X_test, y_:Y_test})
        print('step =',step,'cre =',cre,'acc = ',acc)
        

step = 0 cre = 30.768297 acc =  0.9664
step = 500 cre = 22.829256 acc =  0.9714
step = 1000 cre = 26.88199 acc =  0.9684
step = 1500 cre = 26.990921 acc =  0.978
step = 2000 cre = 24.287539 acc =  0.9774
step = 2500 cre = 19.569044 acc =  0.9786
step = 3000 cre = 25.988615 acc =  0.9636


In [62]:
csv[1:10][['height','weight']]

,height,weight
1,0.710,0.73
2,0.885,0.61
3,0.935,0.48
4,0.765,0.60
5,0.930,0.59
6,0.830,0.53
7,0.965,0.56
8,0.920,0.43
9,0.965,0.64
